<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/notebooks/2020Sight_visit_feature_extractions_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# サイトビジット G 検定対策ビデオ教材

# 画像処理における特徴量抽出の実演教材

- 浅川伸一
- date: 2020-0909
- revised at: 2021-0526

SIFT と SURF が opencv では対応しなくなったため，代わりに ORB を使ったデモを追加しました。




In [ ]:
#最新の OpenCV では surf などが著作権の問題で実装はされなくなりました。
#従って，以前のバージョンをインストールしなければなりません 
#!pip install opencv-python==3.4.2.16
#!pip install opencv-contrib-python==3.4.2.16

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#ファイルをアップロードします
from google.colab import files
#files?

In [ ]:
files.upload()  # ご自身の PC からファイルをアップロードして下さい

In [ ]:
image_file = 'IMG_3189.JPG'  # ファイル名はアップロードしたファイル名に合わせて変更します
img = plt.imread(image_file)
print(img.shape)  # 画像のサイズを表示します

In [ ]:
#OpenCV の読み込み
import cv2

#アップロードしたファイルの表示
img_cv2 = cv2.imread(image_file)
plt.figure(figsize = (12, 16)); plt.axis('off')
plt.imshow(cv2.cvtColor(img_cv2, cv2.COLOR_BGR2RGB))  # OpenCV は RGB ではなく BGR なので変換する


In [ ]:
from skimage.color import rgb2gray

img_gray = rgb2gray(img)
print('画像のサイズ:', img_gray.shape, '\n')

# 2D pixel map
print('画像の画素の値')
print(np.round(img_gray, 2), '\n')

# flattened pixel feature vector
print('一次元にしてみます', (np.round(img_gray.flatten(), 2)))

In [ ]:
plt.figure(figsize = (10, 8)); plt.axis('off') # 表示画像のサイズ単位はインチで横と縦を指定

plt.imshow(img_gray, cmap="gray"); 
plt.show()

plt.figure(figsize = (10, 6)); plt.axis('off')
c_freq, c_bins, c_patches = plt.hist(img_gray.flatten(), bins=30)

In [ ]:
#DoG 関数の定義
def DoG(img, ker1=3, ker2=5):
    """img : cv2.imread file"""

    blur2 = cv2.GaussianBlur(img, (ker2, ker2), 0)
    blur1 = cv2.GaussianBlur(img, (ker1, ker1), 0)

    return blur2 - blur1

In [ ]:
dog_img = DoG(img)
plt.figure(figsize = (10, 8)); plt.axis('off')
plt.imshow(dog_img); plt.show()


dog_img = DoG(img, ker1=3, ker2=9)
plt.figure(figsize = (10, 8)); plt.axis('off')
plt.imshow(dog_img); plt.show()

In [ ]:
#SIFT と SURF の定義
#sift_detector = cv2.xfeatures2d.SIFT_create()
#surf_detector = cv2.xfeatures2d.SURF_create()
#help(cv2.xfeatures2d)


# 注記: 2021年５月２６日

ビデオでは SIFT と SURF を紹介しました。ですが，SIFT と SURF は著作権の関係から opencv ライブラリでは使えなくなりました。
代わりとして ORB を紹介します。
ORB は SIFT や ORB の代替として更に性能が高いように見受けられます。

参考文献:
Rublee, E.,  Rabaud, V., Konolige, K., and Bradski, G (2011) ORB: an efficient alternative to SIFT or SURF, Proceedings IEEE International Conference on Computer Vision.

In [ ]:
# Initiate ORB detector
orb = cv2.ORB_create()
# find the keypoints with ORB
orb_keypoints = orb.detect(img_cv2,None)
# compute the descriptors with ORB
orb_keypoints, des = orb.compute(img_cv2, orb_keypoints)

radius = 15
for ky in orb_keypoints:
    cv2.circle(img_cv2, (int(ky.pt[0]),int(ky.pt[1])), radius, (0,255,0), -1)

plt.figure(figsize = (12, 16))
plt.axis('off')
plt.imshow(cv2.cvtColor(img_cv2, cv2.COLOR_BGR2RGB))

# draw only keypoints location,not size and orientation
#img2 = cv2.drawKeypoints(img_cv2, orb_keypoints, None, color=(0,255,0), flags=0)

In [ ]:
# #SURF の実演
# img_cv2 = cv2.imread(image_file)
# surf_keypoints, _ = surf_detector.detectAndCompute(img_cv2, None)

# radius = 5
# for k in surf_keypoints:
#     cv2.circle(img_cv2, (int(k.pt[0]),int(k.pt[1])), radius, (0,255,0),-1)

# plt.figure(figsize = (6, 8))
# plt.imshow(cv2.cvtColor(img_cv2, cv2.COLOR_BGR2RGB))


In [ ]:
# #sift の実演
# img_cv2 = cv2.imread(image_file)
# sift_keypoints, _ = sift_detector.detectAndCompute(img_cv2, None)

# radius = 5
# for k in sift_keypoints:
#     cv2.circle(img_cv2, (int(k.pt[0]),int(k.pt[1])), radius, (0,255,0),-1)

# plt.figure(figsize = (6, 8)) 
# plt.imshow(cv2.cvtColor(img_cv2, cv2.COLOR_BGR2RGB))


In [ ]:
# Canny アルゴリズムによるエッジ（端点）検出の実演
from skimage.feature import canny

img_edges = canny(img_gray, sigma=3)

plt.figure(figsize = (10, 8)); plt.axis('off')
plt.imshow(img_edges, cmap='binary')


In [ ]:
#HOG の実演
from skimage.feature import hog
from skimage import exposure

# 下の orientation の値は 0 から 9 までです。変化させて，結果を観察してください
fd_img, img_hog = hog(img_gray, orientations=8, pixels_per_cell=(8, 8), 
                      cells_per_block=(3, 3), visualize=True)

# rescaling intensity to get better plots
img_hogs = exposure.rescale_intensity(img_hog, in_range=(0, 0.04))

#plt.figure(figsize = (6, 8))
plt.figure(figsize = (10,8)); plt.axis('off')
plt.imshow(img_hogs, cmap='binary')
